# Introdução
O presente projeto possui como objetivo realizar a classificação de palavras de um Dataset com opiniões de produtos das Lojas Americanas. Partindo de uma lista de comentários com notas iniciais de 1 a 5 estrelas para cada produto, treinaremos nosso modelo de classificação de texto para correlacionar as tokens e frases dos respectivos comentários a uma **categoria final positiva ou negativa**.

# Dataset
O Dataset de avaliações de produtos do site americanas.com.br encontra-se em `./Bases/dados_americanas.xlsx`, e contém as seguintes colunas: 
* `submission_date` (data de submissão) 
* `reviewer_id` (id do usuário) 
* `product_id` (id do produto)
* `product_name` (nome do produto)
* `product_brand` (marca do produto)
* `site_category_lv1` e `site_category_lv2` (categorias do produto)
* `review_title` (título do comentário)
* `overall_rating` (classificação de 1 a 5)
* `recommend_to_a_friend` (recomenda a um amigo)
* `review_text` (texto do comentário)
* `reviewer_birth_year` (ano de nascimento do usuário)
* `reviewer_gender` (sexo do usuário)
* `reviewer_state` (estado do usuário)

Para o nosso estudo, apenas as colunas `review_text` e `overall_rating` são suficientes. Nosso repositório também possui um dataset com as mesmas colunas localizado em `./Bases/teste_dataset.csv`, para que possamos avaliar o modelo criado posteriormente com casos de testes. Os datasets foram obtidos a partir das seguintes fontes:

Dataset Americanas: https://github.com/b2wdigital/b2w-reviews01 (B2W-Reviews01.csv)

Dataset de Teste: https://www.kaggle.com/olistbr/brazilian-ecommerce (olist_order_reviews_dataset.csv)

# Metodologia
Utilizamos uma regressão logística (cujos parâmetros foram escolhidos através do algoritmo GridSearchCV, da biblioteca SKLearn) para classificar os textos em 2 categorias: `0` - Avaliação **Negativa** ou `1` - Avaliação **Positiva**. Primeiramente, foi necessário realizar o pré-processamento dos dados, ou seja, 
normalizar as palavras dos comentários. Esse procedimento incluiu as seguintes acões: 
* Retirar acentos e maiúsculas das palavras iniciais, para convergir palavras escritas de forma diferente num mesmo significado;
* Substituir emails por um único token indicativo (`_EMAIL_`). Pela definição do problema, a presença de e-mails pode ser relevante para indicar que a avaliação é negativa e, portanto, não excluímos os e-mails, mas sim indicamos que ali foi inserido um;
* Substituir pontuações por espaços para evitar junção de palavras. Por exemplo, se um comentário incluísse a frase `adorei,muito bom`, o algoritmo criaria uma palavra `adorei,muito`. Após aplicar a substituição, o resultado da frase seria `adorei muito bom`, permitindo utilizar 3 tokens separados;
* Da mesma forma, podem ser desconsiderados números e caracteres especiais pois seus significados variam muito de acordo com a frase em análise;
* Tokenização, para quebrar o texto em uma lista de palavras;
* Remoção de "letras soltas" (`a`, `o`, `e`, etc.), pois fazem pouca ou nenhuma diferença no resultado final, e podem surgir devido à remoção de algum caractere especial (por exemplo, a remoção do cifrão em valores monetários faz com que uma letra r fique isolada no texto). Geralmente são conjunções ou preposições;
* Ajustar palavras com letras a mais (`Ruim demaissss` se torna igual à `Ruim demais`);
* Substituir abreviações por palavras equivalentes;
* Remoção de stopwords. Ex.: `as`, `e`, `os`, `de`, `para`, `com`, `sem`, `foi`;
* Aplicação de algoritmo de Stemming para lingua portuguesa para evitar que variações de uma mesma palavra sejam identificadas como diferentes..

Nessa etapa, também **removemos todos os comentários com notas igual à 3**, pois eles representavam tanto avaliações muito ruins quanto muito boas. Uma possibilidade que explica isso é que o site automaticamente utilize a nota 3 caso o usuário não forneça alguma manualmente. Assim, classificamos as notas 1 e 2 na categoria `0 - Avaliação Negativa`, e as notas 4 e 5 em `1 - Avaliação Positiva`. Também **retiramos diversas colunas do Dataset**, mantendo apenas as necessárias: `review_text` (renomeada para `Texto`) e a `Classe` binária que criamos.

Após isso, obtemos os textos pré-processados e os retornamos. Por exemplo, 
> MEU FILHO AMOU! PARECE DE VERDADE

tornou-se

> filh amou parec verdad

Realizamos o pré processamento da base e salvamos o resultado em um arquivo `xlsx`, `./Bases/Base_Final.xlsx`, e então aplicamos a regressão logística. 

## Regressão Logística

Utilizando a vetorização utilizando a métrica [TFIDF](https://www.freecodecamp.org/news/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3/) ("Term frequency - Inverse Data Frequency"), que representa os textos na forma de um vetor com o tf-idf das palavras, bigramas e trigramas, fomos capazes de realizar o treinamento do algoritmo de **regressão logística** realizando a tunagem dos parâmetros por meio de uma GridSearchCV, por meio da qual encontramos um melhor estimador (estimador com maior F1 score). Para auxiliar com possíveis pré-carregamentos, salvamos o modelo no caminho `./Auxiliar/modelo.pkl` e desenvolvemos uma função para retornar as predições como "Avaliação Positiva" ou "Avaliação Negativa" ao invés de "1" ou "0", facilitando a visualização da resposta para qualquer frase arbitrária. Também verificamos, por meio do modelo, **quais palavras possuem maior correlação** com as categorias positiva e negativa.

Utilizando um dataset de teste, foi possível também testar o modelo nos comentários e verificar qual classe deveria ter sido escolhida. Isso nos permitiu **calcular acurácia, F1, AUC e criar uma matriz de confusão** para os dados de teste.

# Código
Durante a criação do código, importamos as bibliotecas `numpy` e `nltk` (Natural Language Toolkit), além do `sklearn` (Scikit Learn) e `pandas`. Para o `nltk`, utilizamos os `stopwords`, `punkt` (pontuação) e `rslp` (o Stemmer "Removedor de Sufixos da Lingua Portuguesa").
Também utilizamos um corpus das stopwords em português com adição dos meses, e abreviações específicas para o idioma português, pois alguns autores de comentários optaram por escrever palavras abreviadamente.

Uma busca exaustiva para um estimador foi feita com o método [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) do `sklearn`, que cria uma pipeline (sequência de métodos) com `tfidf` e `classifier`. Por padrão utilizamos 4 cores, mas o valor `-1` permite paralelismo ótimo de acordo com a configuração do computador que executa o programa. Quando executado em um Docker container, houve a necessidade de aumentar a memória para evitar erros durante a execução.

# Conclusões
A geração do modelo foi a parte mais custosa em termos computacionais do trabalho, podendo levar mais de 10 minutos. Porém após essa etapa obtemos resultados compensatórios: o modelo obteve acurácia, F1 e AUC > 0.9 no conjunto de teste. Ele foi especialmente capaz de detectar avaliações positivas com alta precisão.

In [1]:
#Importação de bibliotecas, download de recursos e definição de variáveis globais (stopwords e dicionario para corrigir abreviações)

#!pip install os
import os
#!pip install pandas
import pandas as pd
#!pip install numpy
import numpy as np
#!pip install nltk
import nltk
#!pip install re
import re
#!pip install unicodedata
import unicodedata
#!pip install joblib
import joblib
#!pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
stopwords = nltk.corpus.stopwords.words('portuguese') + ['outubro', 'novembro', 'dezembro', 'janeiro', 'fevereiro', 'marco', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', ' ']
abreviacoes = {'mt':'muito', 'mto':'muito', 'dms':'demais', 'fds':'fim de semana', 'blz':'beleza', 'bls':'beleza'}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pichau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pichau\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Pichau\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
#Definição de funções para pré-processamento

stemmer = nltk.stem.RSLPStemmer()
def stemming(lista):
    retorno = []
    for word in lista:
        retorno.append(stemmer.stem(word))
    return retorno

def ajusta_abreviacoes(lista):
    retorno = []
    for word in lista:
        if word in list(abreviacoes.keys()):
            retorno.append(abreviacoes[word])
        else:
            retorno.append(word)
    return retorno

def ajusta_letras(lista):
    retorno = []
    for word in lista:
        palavra = list(word)
        palavra_certa = []
        for i in range(len(palavra)):
            letra = palavra[i]
            if i == len(palavra)-1:
                letra_prox='_FIM_'
            else:
                letra_prox = palavra[i+1]
            
            if letra != letra_prox or letra in ['s', 'r']:
                palavra_certa.append(letra)
        retorno.append(''.join(palavra_certa))
            
    return retorno

    
def preprocessamento(texto):
    #Removendo acentos e maiúsculas...
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII','ignore').decode('ASCII').lower()
    #Substituindo e-mails por token indicativo...
    texto = re.sub(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '_EMAIL_', texto)
    #Substituindo pontuações por espaços (evitar encavalamento de palavras)...
    texto = re.sub(r'[,.;:\(\)]', ' ', texto)
    #Removendo numeros e caracteres especiais...
    texto = re.sub(r'[^a-z\s]', '', texto)
    #Tokenização...
    tokens = nltk.tokenize.word_tokenize(texto)
    #Removendo letras soltas...
    tokens = [token for token in tokens if len(token) > 1]
    #Ajustando palavras com letras a mais:
    tokens = ajusta_letras(tokens)
    #Ajustando abreviações...
    tokens = ajusta_abreviacoes(tokens)
    #Removendo stopwords...
    tokens_sw = [token for token in tokens if token not in stopwords]
    #Aplicando Stemming RLSP
    tokens_sw = stemming(tokens_sw)
    return ' '.join(tokens_sw)

def preprocessarbase(base, coluna): #Executa o pré-processamento a todos os textos de uma base de dados
    base[coluna] = base[coluna].apply(preprocessamento)
    return None

In [3]:
#Reconhecimento dos caminhos utilizados e importação da base

path = os.getcwd() + '\\'
path_base = path + 'Bases\\'
path_auxiliar = path+ 'Auxiliar\\'
base_dados = pd.read_excel(path_base + 'dados_americanas.xlsx')

base_dados.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965.0,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178.0,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329.0,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581.0,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803.0,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


In [4]:
#Tratamento da base e geração de classes binárias

base_dados = base_dados[base_dados['overall_rating'] != 3].reset_index(drop=True)
base_dados['Classe'] = np.where(base_dados['overall_rating'] > 3, 1, 0)

base_dados.drop(['submission_date', 'reviewer_id', 'product_id', 'product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'review_title', 'overall_rating', 'recommend_to_a_friend', 'reviewer_birth_year', 'reviewer_gender', 'reviewer_state'],axis=1,inplace=True)
base_dados.rename(columns={'review_text':'Texto'}, inplace=True)

base_dados #retorna a base já tratada

,Texto,Classe
0,Estou contente com a compra entrega rápida o ú...,1
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1
4,"A entrega foi no prazo, as americanas estão de...",1
...,...,...
116053,"Vale muito, estou usando no controle do Xbox e...",1
116054,"Prático e barato, super indico o produto para ...",1
116055,Chegou antes do prazo previsto e corresponde a...,1
116056,"Material fraco, poderia ser melhor. Ficou deve...",0


In [5]:
#Execução do pré-processamento dos textos da base, com visualização do tempo gasto

from time import time
print("Inicio do pré-processamento dos dados...")
inicio = time()
base_preproc = base_dados.copy(deep=True)
preprocessarbase(base_preproc, 'Texto')
fim = time()
print("Pré-processamento finalizado. Tempo decorrido: " + str(fim-inicio) + " segundos.")

Inicio do pré-processamento dos dados...
Pré-processamento finalizado. Tempo decorrido: 89.96394896507263 segundos.


In [6]:
#Envio da base pré-processada para um arquivo excel
base_preproc.to_excel(path_base+'Base_Final.xlsx')
base_preproc #retorna a base pré-processada

,Texto,Classe
0,cont compr entreg rap unic problem americ troc...,1
1,apen consegu compr lind cop acril,1
2,sup agil pratic outr panel eletr costum us out...,1
3,filh amou parec verdad tant detalh,1
4,entreg praz americ esta parab smart tv boa nav...,1
...,...,...
116053,val us control xbox dur seman carg par jog tod...,1
116054,pra barat sup indic produt corr dia dia praz e...,1
116055,cheg ant praz previst correspond anunci,1
116056,mater frac pod ser melhor fic dev opinia,0


In [7]:
#Importação de funções específicas da biblioteca sklearn, definição de parâmetros para busca exaustiva (GridSearchCV) 
#e realização da mesma

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid ={
    'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 10),
    'classifier__solver' : ['sag', 'lbfgs'],
}

pipe = Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=20, ngram_range=(1,3))),
                       ('classifier', LogisticRegression(max_iter = 200000))])

gridsearch = GridSearchCV(pipe, param_grid=param_grid, cv = 3, n_jobs=4, scoring='f1', verbose=3)

gridsearch.fit(base_preproc['Texto'], base_preproc['Classe'])

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   38.4s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:  6.9min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(min_df=20,
                                                        ngram_range=(1, 3))),
                                       ('classifier',
                                        LogisticRegression(max_iter=200000))]),
             n_jobs=4,
             param_grid={'classifier__C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04]),
                         'classifier__penalty': ['l1', 'l2'],
                         'classifier__solver': ['sag', 'lbfgs']},
             scoring='f1', verbose=3)

In [8]:
#Escolha do melhor modelo (de acordo com o F1 score) e salvamento dele em arquivo .pkl para agilizar eventuais testes etc.

model = gridsearch.best_estimator_
joblib.dump(model, path_auxiliar+'modelo.pkl')

['C:\\Users\\Pichau\\Documents\\Faculdade\\Projeto2_PLN\\Auxiliar\\modelo.pkl']

In [9]:
#Visualização da correlação de palavras/termos com as classes positiva (maiores coeficientes) e negativa (menores coeficientes)

sorted_coef_index = model['classifier'].coef_[0].argsort()
feature_names = np.array(model['tfidf'].get_feature_names())

print('Menores coeficientes:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Maiores coeficientes: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))
print('F1 Score do modelo: \n{}\n'.format(gridsearch.best_score_))



Menores coeficientes:
['nao recom' 'nao' 'pess' 'nao func' 'ruim' 'nao gost' 'horri' 'decepcion'
 'frac' 'insatisfeit']

Maiores coeficientes: 
['otim' 'excel' 'recom' 'ador' 'bom' 'perfeit' 'satisfeit' 'ame'
 'maravilh' 'gost']

F1 Score do modelo: 
0.9631607919291493



In [10]:
#Função para traduzir 0 ou 1 em Avaliação Negativa ou Avaliação Positiva

dic_modelo = {0:'Avaliação Negativa', 1:'Avaliação Positiva'}
def avaliacao_produto(frase):
    if type(frase) == str:
        return dic_modelo[model.predict([preprocessamento(frase)])[0]]
    else:
        retorno = ''
        preproc = pd.DataFrame(columns=['Texto'])
        preproc['Texto'] = frase
        preprocessarbase(preproc, 'Texto')
        predicoes = model.predict(preproc['Texto'])
        for predicao in predicoes:
            retorno += dic_modelo[predicao] + '\n'
        return retorno
    

In [17]:
#Teste da função definida acima
print(avaliacao_produto('Adorei o produto!'))

Avaliação Positiva


In [12]:
#Importação e tratamento de base para testagem do modelo (para facilitar o cálculo das métricas, utilizamos as predições
#diretas modelo, em 0 e 1, ao invés de passá-las pela tradução)

teste = pd.read_csv(path_base+'teste_dataset.csv').drop(columns=['review_id', 'order_id', 'review_comment_title', 'review_creation_date', 'review_answer_timestamp'], axis=1).dropna(axis=0).reset_index(drop=True)
teste = teste[teste['review_score'] != 3].rename(columns={'review_comment_message': 'Texto'})

teste['Classificacao'] = np.where(teste['review_score']>3, 1, 0)
teste.drop('review_score', axis=1, inplace=True)
preprocessarbase(teste, 'Texto')
predict = model.predict(teste['Texto']) #armazena predições do modelo para a base de teste na variável "predict"
real = teste['Classificacao']

In [13]:
#Função para testar o modelo com base nas classes reais e nas preditas. Exibe diferentes métricas (acurácia, F1 score e AUC 
#score) e a matriz de confusão do modelo

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score

def matriz_confusão(real, pred):
    retorno = pd.DataFrame(confusion_matrix(real, pred), index=['Real: Avaliação Negativa', 'Real: Avaliação Positiva'])
    retorno.rename(columns={0:'Pred: Avaliação Negativa', 1:'Pred: Avaliação Positiva'}, inplace=True)
    print('A acurácia do modelo nos testes foi de {}\n'.format(accuracy_score(real, pred)))
    print('O F1 Score do modelo nos testes foi de {}\n'.format(f1_score(real, pred)))
    print('O AUC Score do modelo nos testes foi de {}\n'.format(roc_auc_score(real, pred)))
    return retorno

In [14]:
#Executa a função acima com os dados da base de teste
matriz_confusão(real, predict)

A acurácia do modelo nos testes foi de 0.9257246376811594

O F1 Score do modelo nos testes foi de 0.9465934190406071

O AUC Score do modelo nos testes foi de 0.9163750785897374



,Pred: Avaliação Negativa,Pred: Avaliação Positiva
Real: Avaliação Negativa,10188,1220
Real: Avaliação Positiva,1609,25071
